# 🚀 RankSEG with PaddleSeg

This notebook demonstrates how to integrate **RankSEG** with **PaddleSeg**, an end-to-end high-efficiency image segmentation development kit based on PaddlePaddle.

RankSEG can be easily plugged into the inference pipeline of PaddleSeg models to optimize the segmentation results for metrics like Dice and IoU.

**Links:**
- [RankSEG GitHub](https://github.com/rankseg/rankseg)
- [PaddleSeg GitHub](https://github.com/PaddlePaddle/PaddleSeg)

## 1. Install Dependencies
Install `paddleseg` and `rankseg`.

In [ ]:
!pip install paddlepaddle-gpu
!pip install paddleseg
!pip install rankseg

## 2. Load Pre-trained Model
We will load a PP-LiteSeg model trained on Cityscapes.

In [ ]:
import paddle
from paddleseg.models import PPLiteSeg
from paddleseg.transforms import Compose, Resize, Normalize
import numpy as np
import matplotlib.pyplot as plt
import cv2
import requests
from io import BytesIO
from PIL import Image

# Download demo image
url = "https://raw.githubusercontent.com/PaddlePaddle/PaddleSeg/release/2.8/docs/images/cityscapes_demo.png"
response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("RGB")
img.save("cityscapes_demo.png")

# Initialize Model
model = PPLiteSeg(num_classes=19)
# Load pretrained weights (placeholder link, assumes available or download manually)
# For demo purposes, we'll initialize random weights or skip loading if URL unavailable
# model_url = 'https://paddleseg.bj.bcebos.com/dygraph/cityscapes/pp_liteseg_stdc1_cityscapes_1024x512_scale0.5_160k/model.pdparams'
# param_state_dict = paddle.utils.download.get_weights_from_url(model_url)
# model.set_dict(param_state_dict)
model.eval()

print("Model loaded.")

## 3. Preprocessing & Inference
PaddleSeg uses `transforms` to prepare the data. We need to get the logits (probability map) from the model.

In [ ]:
transforms = Compose([
    Resize(target_size=(512, 1024)),
    Normalize()
])

# Preprocess
data = transforms({'img': np.array(img)}) 
img_tensor = paddle.to_tensor(data['img']).unsqueeze(0) # Add batch dim

# Inference
with paddle.no_grad():
    logits = model(img_tensor)[0] # PaddleSeg models return a list
    probs = paddle.nn.functional.softmax(logits, axis=1)

print(f"Logits shape: {logits.shape}")

## 4. ⚡ Apply RankSEG
Since RankSEG is built on PyTorch, we need to convert Paddle tensors to PyTorch tensors (or NumPy arrays) to use it. RankSEG accepts NumPy arrays as well (internally converting to Torch), or we can manually convert.

**Note:** RankSEG is optimized for PyTorch. If you are using Paddle, passing NumPy arrays to RankSEG is the easiest way.

In [ ]:
from rankseg import RankSEG
import torch

# Convert Paddle tensor to Numpy
probs_np = probs.numpy()

# Convert to PyTorch tensor (RankSEG expects Torch tensors)
probs_torch = torch.from_numpy(probs_np)

# Apply RankSEG
rankseg = RankSEG(metric='dice', solver='RMA')
preds = rankseg.predict(probs_torch)

# Convert back to Numpy for visualization
rankseg_result = preds.numpy()[0]
base_result = paddle.argmax(probs, axis=1).numpy()[0]

print("RankSEG optimization complete.")

## 5. Visual Comparison

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(1, 3, 1)
plt.imshow(img)
plt.title("Input")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(base_result)
plt.title("PaddleSeg Argmax")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(rankseg_result)
plt.title("RankSEG Optimized")
plt.axis('off')

plt.show()